# 02d - TabPFN-TS with Calendar Features

Demonstrates how adding simple calendar features transforms TabPFN-TS performance.

**Hypothesis:** TabPFN-TS underperforms on univariate data because it's designed for multivariate tabular forecasting. Adding calendar features should unlock its true potential.

**Enhancement:**
- **Day of week** (captures weekly seasonality)
- **Day of year** (captures annual seasonality)
- **Month** (categorical time signal)
- **Is weekend** (binary indicator)
- **Fourier terms** (sin/cos pairs for 7-day and 365-day cycles)

**Comparison:**
- Baseline: TabPFN-TS (univariate only, from nb/02_roll_loop.ipynb)
- Enhanced: TabPFN-TS + calendar features (this notebook)

## 0. Setup

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Model imports
from autogluon.timeseries import TimeSeriesDataFrame
from tabpfn_time_series import TabPFNTimeSeriesPredictor, TabPFNMode, FeatureTransformer
from tabpfn_time_series.features import RunningIndexFeature, CalendarFeature, AutoSeasonalFeature

In [3]:
# Set random seed
np.random.seed(42)

In [4]:
# Define paths
project_root = Path.cwd().parent
data_dir = project_root / 'data'
results_dir = project_root / 'results' / 'forecasts'
results_dir.mkdir(parents=True, exist_ok=True)

### Load Data

In [5]:
# Load cleaned time series
ts = pd.read_pickle(data_dir / 'flu_daily_clean.pkl')
print(f"Loaded data: {ts.shape[0]} observations")
print(f"Date range: {ts.index.min()} to {ts.index.max()}")
print(f"Frequency: {ts.index.freq}")

Loaded data: 1078 observations
Date range: 2022-07-04 00:00:00 to 2025-06-15 00:00:00
Frequency: <Day>


### Configure Rolling Windows

In [6]:
# Same configuration as nb/02_roll_loop.ipynb for fair comparison
HORIZONS = [7, 28]
ORIGINS = pd.date_range('2024-07-08', '2025-05-26', freq='2W-MON')
MIN_TRAIN = 730  # 2 years minimum training data

print(f"Forecast horizons: {HORIZONS}")
print(f"Number of forecast origins: {len(ORIGINS)} (bi-weekly)")
print(f"Minimum training days: {MIN_TRAIN}")
print(f"Total forecasts: {len(ORIGINS) * len(HORIZONS)}")

Forecast horizons: [7, 28]
Number of forecast origins: 24 (bi-weekly)
Minimum training days: 730
Total forecasts: 48


### Initialize TabPFN Client

In [7]:
from dotenv import load_dotenv
import os
import tabpfn_client

# Load API key from .env
env_file = project_root / ".env"
if env_file.exists():
    load_dotenv(env_file)
    api_key = os.getenv("PRIORLABS_API_KEY")
    if api_key:
        tabpfn_client.set_access_token(api_key)
        print("✅ TabPFN client initialized")
    else:
        print("⚠️  PRIORLABS_API_KEY not found in .env")
else:
    print("⚠️  .env file not found")

✅ TabPFN client initialized


## 1. Official TabPFN Feature Engineering

TabPFN provides specialized features optimized for time series forecasting:
- **RunningIndexFeature**: Time index for trend extrapolation
- **CalendarFeature**: Sine/cosine transformations for cyclical calendar patterns
- **AutoSeasonalFeature**: DFT-based adaptive seasonality detection

In [8]:
# Initialize feature transformer with official TabPFN features
selected_features = [
    RunningIndexFeature(),      # Adds time index (0, 1, 2, ...) for trend extrapolation
    CalendarFeature(),           # Sine/cosine encoding of calendar components
    AutoSeasonalFeature(),       # DFT-based seasonality detection from training data
]

feature_transformer = FeatureTransformer(selected_features)

print("✅ FeatureTransformer initialized with:")
for feat in selected_features:
    print(f"   - {feat.__class__.__name__}")

✅ FeatureTransformer initialized with:
   - RunningIndexFeature
   - CalendarFeature
   - AutoSeasonalFeature


### Test Feature Transformer

In [9]:
# Test on a small sample
test_df = pd.DataFrame({
    'timestamp': pd.date_range('2024-01-01', periods=10, freq='D'),
    'target': np.random.rand(10),
    'item_id': 'test'
})

test_tsdf = TimeSeriesDataFrame.from_data_frame(
    test_df,
    id_column='item_id',
    timestamp_column='timestamp'
)

# Create test data (forecast period)
test_forecast_df = pd.DataFrame({
    'timestamp': pd.date_range('2024-01-11', periods=5, freq='D'),
    'target': np.nan,
    'item_id': 'test'
})

test_forecast_tsdf = TimeSeriesDataFrame.from_data_frame(
    test_forecast_df,
    id_column='item_id',
    timestamp_column='timestamp'
)

# Apply feature transformer
train_with_features, test_with_features = feature_transformer.transform(test_tsdf, test_forecast_tsdf)

print("Original columns:", list(test_tsdf.columns))
print(f"\nColumns after FeatureTransformer: {list(train_with_features.columns)}")
print(f"Total features added: {len(train_with_features.columns) - 1}")  # -1 for 'target'

print("\nSample with features:")
print(train_with_features.head())

Original columns: ['target']

Columns after FeatureTransformer: ['target', 'running_index', 'year', 'second_of_minute_sin', 'second_of_minute_cos', 'minute_of_hour_sin', 'minute_of_hour_cos', 'hour_of_day_sin', 'hour_of_day_cos', 'day_of_week_sin', 'day_of_week_cos', 'day_of_month_sin', 'day_of_month_cos', 'day_of_year_sin', 'day_of_year_cos', 'week_of_year_sin', 'week_of_year_cos', 'month_of_year_sin', 'month_of_year_cos', 'sin_#0', 'cos_#0', 'sin_#1', 'cos_#1', 'sin_#2', 'cos_#2', 'sin_#3', 'cos_#3', 'sin_#4', 'cos_#4']
Total features added: 28

Sample with features:
                      target  running_index  year  second_of_minute_sin  \
item_id timestamp                                                         
test    2024-01-01  0.374540              0  2024                   0.0   
        2024-01-02  0.950714              1  2024                   0.0   
        2024-01-03  0.731994              2  2024                   0.0   
        2024-01-04  0.598658              3  2024

## 2. TabPFN-TS with Features

In [10]:
def forecast_tabpfn_enhanced(train_series, horizon, feature_transformer, item_id='flu_positivity'):
    """
    Forecast with TabPFN-TS using official FeatureTransformer.

    Parameters
    ----------
    train_series : pd.Series
        Training data (datetime-indexed)
    horizon : int
        Number of steps ahead to forecast
    feature_transformer : FeatureTransformer
        Initialized FeatureTransformer with selected features
    item_id : str
        Identifier for the time series

    Returns
    -------
    dict : {'q0.1': float, 'q0.5': float, 'q0.9': float}
        Forecast quantiles
    """
    # Prepare training data as TimeSeriesDataFrame
    df_prep = train_series.reset_index()
    df_prep.columns = ['timestamp', 'target']
    df_prep['item_id'] = item_id

    train_tsdf = TimeSeriesDataFrame.from_data_frame(
        df_prep,
        id_column='item_id',
        timestamp_column='timestamp'
    )

    # Create test dates (forecast horizon)
    forecast_start = train_series.index[-1] + pd.Timedelta(days=1)
    test_dates = pd.date_range(forecast_start, periods=horizon, freq='D')

    test_df = pd.DataFrame({
        'timestamp': test_dates,
        'item_id': item_id,
        'target': np.nan
    })

    test_tsdf = TimeSeriesDataFrame.from_data_frame(
        test_df,
        id_column='item_id',
        timestamp_column='timestamp'
    )

    # Apply feature transformer
    train_tsdf, test_tsdf = feature_transformer.transform(train_tsdf, test_tsdf)

    # Initialize predictor and forecast
    predictor = TabPFNTimeSeriesPredictor(tabpfn_mode=TabPFNMode.CLIENT)
    pred = predictor.predict(train_tsdf, test_tsdf)

    # Extract final horizon prediction
    pred_slice = pred.loc[item_id]

    return {
        'q0.1': float(pred_slice[0.1].iloc[-1]),
        'q0.5': float(pred_slice[0.5].iloc[-1]),
        'q0.9': float(pred_slice[0.9].iloc[-1])
    }

### Test TabPFN-TS with Official Features

In [11]:
# Test on a single rolling window
test_origin = pd.Timestamp('2024-06-01')
test_horizon = 7

# Get training data
train = ts[ts.index < test_origin]
print(f"Train size: {len(train)} days")
print(f"Train range: {train.index.min()} to {train.index.max()}")

# Generate forecast
print("\nTesting TabPFN-TS with official FeatureTransformer (may take a few seconds)...")
pred = forecast_tabpfn_enhanced(train, test_horizon, feature_transformer)

target_date = test_origin + pd.Timedelta(days=test_horizon - 1)
print(f"\nForecast for {target_date}:")
print(f"  q0.1: {pred['q0.1']:.2f}%")
print(f"  q0.5: {pred['q0.5']:.2f}%")
print(f"  q0.9: {pred['q0.9']:.2f}%")

# Compare to actual
if target_date in ts.index:
    actual = ts.loc[target_date]
    print(f"\nActual: {actual:.2f}%")
    print(f"Within 80% interval: {pred['q0.1'] <= actual <= pred['q0.9']}")

Train size: 698 days
Train range: 2022-07-04 00:00:00 to 2024-05-31 00:00:00

Testing TabPFN-TS with official FeatureTransformer (may take a few seconds)...


Predicting time series: 100%|██████████| 1/1 [00:00<00:00, 123.76it/s]
Processing: 100%|██████████| [00:03<00:00]



Forecast for 2024-06-07 00:00:00:
  q0.1: 0.50%
  q0.5: 0.74%
  q0.9: 0.97%

Actual: 0.90%
Within 80% interval: True


## 3. Rolling Forecast Loop

In [12]:
def run_rolling_forecasts_enhanced(ts, origins, horizons, min_train, feature_transformer):
    """
    Run rolling forecasts for TabPFN-TS with official FeatureTransformer.

    Parameters
    ----------
    ts : pd.Series
        Full time series
    origins : pd.DatetimeIndex
        Forecast origin dates
    horizons : list of int
        Forecast horizons
    min_train : int
        Minimum training window size
    feature_transformer : FeatureTransformer
        Initialized FeatureTransformer with selected features

    Returns
    -------
    pd.DataFrame
        Forecast results
    """
    results = []

    # Progress bar
    total_iter = len(origins) * len(horizons)
    pbar = tqdm(total=total_iter, desc="TabPFN-TS Enhanced rolling forecasts")

    for origin in origins:
        # Get training data (all data before origin)
        train = ts[ts.index < origin]

        # Skip if insufficient training data
        if len(train) < min_train:
            continue

        for horizon in horizons:
            # Target forecast date
            target_date = origin + pd.Timedelta(days=horizon - 1)

            # Skip if target date is beyond available data
            if target_date not in ts.index:
                pbar.update(1)
                continue

            # Get actual value
            actual = ts.loc[target_date]

            # TabPFN-TS Enhanced
            try:
                pred = forecast_tabpfn_enhanced(train, horizon, feature_transformer)
                results.append({
                    'date': target_date,
                    'origin': origin,
                    'horizon': horizon,
                    'model': 'TabPFN_Enhanced',
                    'q0.1': pred['q0.1'],
                    'q0.5': pred['q0.5'],
                    'q0.9': pred['q0.9'],
                    'actual': actual
                })
            except Exception as e:
                print(f"\n⚠️  Failed forecast for origin={origin}, horizon={horizon}: {e}")
                pass  # Skip failed forecasts

            pbar.update(1)

    pbar.close()

    return pd.DataFrame(results)

### Execute Rolling Forecasts

**Warning:** This will make API calls and take several minutes.

In [13]:
print("Starting TabPFN-TS Enhanced rolling forecast generation...")
print(f"Total iterations: {len(ORIGINS) * len(HORIZONS)}")
print("\nUsing official TabPFN features:")
print("  - RunningIndexFeature (trend/extrapolation)")
print("  - CalendarFeature (cyclical encoding)")
print("  - AutoSeasonalFeature (DFT-based seasonality)")
print("\nThis will take approximately 5-8 minutes (API calls)...\n")

forecast_results = run_rolling_forecasts_enhanced(
    ts=ts,
    origins=ORIGINS,
    horizons=HORIZONS,
    min_train=MIN_TRAIN,
    feature_transformer=feature_transformer
)

print(f"\n✅ Rolling forecasts complete! Generated {len(forecast_results)} forecasts.")

Starting TabPFN-TS Enhanced rolling forecast generation...
Total iterations: 48

Using official TabPFN features:
  - RunningIndexFeature (trend/extrapolation)
  - CalendarFeature (cyclical encoding)
  - AutoSeasonalFeature (DFT-based seasonality)

This will take approximately 5-8 minutes (API calls)...



Predicting time series: 100%|██████████| 1/1 [00:00<00:00, 782.52it/s]?it/s]
Processing: 100%|██████████| [00:03<00:00]
Predicting time series: 100%|██████████| 1/1 [00:00<00:00, 1554.02it/s]1,  8.33s/it]
Processing: 100%|██████████| [00:04<00:00]
Predicting time series: 100%|██████████| 1/1 [00:00<00:00, 1038.45it/s]6,  7.09s/it]
Processing: 100%|██████████| [00:04<00:00]
Predicting time series: 100%|██████████| 1/1 [00:00<00:00, 1402.31it/s]3,  6.98s/it]
Processing: 100%|██████████| [00:04<00:00]
Predicting time series: 100%|██████████| 1/1 [00:00<00:00, 1395.78it/s]5,  7.17s/it]
Processing: 100%|██████████| [00:04<00:00]
Predicting time series: 100%|██████████| 1/1 [00:00<00:00, 1406.54it/s]4,  7.32s/it]
Processing: 100%|██████████| [00:04<00:00]
Predicting time series: 100%|██████████| 1/1 [00:00<00:00, 865.70it/s]54,  7.00s/it]
Processing: 100%|██████████| [00:04<00:00]
Predicting time series: 100%|██████████| 1/1 [00:00<00:00, 1405.13it/s]9,  7.54s/it]
Processing: 100%|██████████


✅ Rolling forecasts complete! Generated 47 forecasts.


## 4. Save Results

In [14]:
# Save forecasts
output_path = results_dir / 'tabpfn_enhanced.parquet'
forecast_results.to_parquet(output_path, index=False)
print(f"✅ Saved {len(forecast_results)} forecasts → {output_path}")

# Display summary
print("\nForecast summary by horizon:")
print(forecast_results.groupby('horizon').size())

✅ Saved 47 forecasts → /home/mikhailarutyunov/projects/time-series-flu/results/forecasts/tabpfn_enhanced.parquet

Forecast summary by horizon:
horizon
7     24
28    23
dtype: int64


## 5. Quick Comparison vs Baseline

In [15]:
# Load baseline TabPFN results for comparison
baseline_path = results_dir / 'tabpfn.parquet'

if baseline_path.exists():
    baseline = pd.read_parquet(baseline_path)

    # Compute quick MAE comparison
    baseline_mae = np.mean(np.abs(baseline['actual'] - baseline['q0.5']))
    enhanced_mae = np.mean(np.abs(forecast_results['actual'] - forecast_results['q0.5']))

    improvement = (baseline_mae - enhanced_mae) / baseline_mae * 100

    print("=" * 60)
    print("QUICK COMPARISON: TabPFN Baseline vs Enhanced")
    print("=" * 60)
    print(f"Baseline (univariate):     MAE = {baseline_mae:.3f}")
    print(f"Enhanced (+ features):     MAE = {enhanced_mae:.3f}")
    print(f"Improvement:               {improvement:+.1f}%")
    print("=" * 60)

    # Coverage comparison
    baseline_cov = np.mean((baseline['actual'] >= baseline['q0.1']) &
                           (baseline['actual'] <= baseline['q0.9'])) * 100
    enhanced_cov = np.mean((forecast_results['actual'] >= forecast_results['q0.1']) &
                           (forecast_results['actual'] <= forecast_results['q0.9'])) * 100

    print(f"\nPrediction Interval Coverage (nominal 80%):")
    print(f"  Baseline: {baseline_cov:.1f}%")
    print(f"  Enhanced: {enhanced_cov:.1f}%")
    print(f"  Improvement: {enhanced_cov - baseline_cov:+.1f} percentage points")
else:
    print("⚠️  Baseline results not found. Run nb/02_roll_loop.ipynb first.")

QUICK COMPARISON: TabPFN Baseline vs Enhanced
Baseline (univariate):     MAE = 2.779
Enhanced (+ features):     MAE = 2.319
Improvement:               +16.5%

Prediction Interval Coverage (nominal 80%):
  Baseline: 59.6%
  Enhanced: 83.0%
  Improvement: +23.4 percentage points


## 6. Checkpoint Summary

**Outcomes:**
- ✅ `results/forecasts/tabpfn_enhanced.parquet` created with 47-48 forecasts
- ✅ Used official TabPFN FeatureTransformer with:
  - **RunningIndexFeature**: Time index for trend extrapolation (missing in manual approach)
  - **CalendarFeature**: Sine/cosine transformations for cyclical patterns
  - **AutoSeasonalFeature**: DFT-based adaptive seasonality (adapts to actual data vs hard-coded)
- ✅ No errors or missing forecasts

**Key Differences from Manual Approach:**
1. **RunningIndexFeature** enables trend extrapolation (critical for forecasting!)
2. **AutoSeasonalFeature** uses DFT to detect actual seasonality in flu data (not assumed 7/365-day cycles)
3. **FeatureTransformer** ensures consistent feature application to train/test sets
4. Expected to significantly improve vs manual features (-4.7% baseline)

**Hypothesis:**
Previous degradation was due to:
- Missing trend/index feature (no extrapolation capability)
- Wrong seasonality assumptions (hard-coded vs DFT-detected)
- Manual features not in TabPFN's expected format

Official approach should unlock TabPFN's true potential on this univariate epidemiological task.

**Next Steps:**
1. Update `nb/03_evaluation.ipynb` to include TabPFN_Enhanced (official features)
2. Compare: TabPFN baseline vs TabPFN_Enhanced (manual) vs TabPFN_Enhanced (official)
3. Update `nb/04_report.ipynb` to visualize the improvement

**For LinkedIn Article:**
> "Official TabPFN features (RunningIndexFeature + CalendarFeature + AutoSeasonalFeature with DFT) improved performance by X%, demonstrating that foundation models need **the right features, not just any features**. The lesson: **feature engineering is alive—it's just more sophisticated now.**"